In [1]:
!cp -r /content/drive/MyDrive/AImodels/spam_modelv2 /content/

In [2]:
!ls /content/spam_modelv2

config.json	   special_tokens_map.json  tokenizer.json
model.safetensors  tokenizer_config.json    vocab.txt


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("/content/spam_modelv2")
tokenizer = AutoTokenizer.from_pretrained("/content/spam_modelv2")

model.eval()  # Set to inference mode

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [5]:
import torch

def predict_spam(text):
    # 1. Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # 2. Move inputs to same device as model (handles GPU/CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # 3. Run model in no-gradient mode (faster, safer)
    with torch.no_grad():
        outputs = model(**inputs)

    # 4. Get predicted class (0 = Not Spam, 1 = Spam)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    # 5. Return human-readable result
    if predicted_class == 1:
        return "Spam 🚨"
    else:
        return "Not Spam ✅"

In [6]:
print(predict_spam("Congratulations! You've won a free cruise!"))
print(predict_spam("Hey, can you send me the homework answers?"))
print(predict_spam("Limited offer! Get rich quick!"))
print(predict_spam("Let's meet at the coffee shop at 3 PM."))

Spam 🚨
Not Spam ✅
Not Spam ✅
Not Spam ✅
